# Sistemas de recomendación y factorización matricial, con Typo Producto

In [1]:
import surprise
import numpy as np
import pandas as pd
import urllib
import io

In [2]:
#Usaremos los datos de los clientes que repiten compras a lo largo del año analizado
data = pd.read_csv('DataTypo_reducido.csv')
#Elimino la columna Unnamed: 0
data.drop(['Unnamed: 0'], axis=1, inplace=True)

dataset = data.rename(columns={"customerId": 'uid', "productId" : 'iid', "purchase_count": "rating"})

dataset.head() 

,uid,iid,rating
0,0,Toallas y Toallones,1
1,0,Toallas y Toallones,1
2,0,Sábanas,1
3,0,Sábanas,1
4,0,Sábanas,1


In [3]:
a = dataset.uid == 2 
dataset[a]

,uid,iid,rating
6,2,Toallas y Toallones,2
7,2,Toallas y Toallones,1
8,2,Toallas y Toallones,1
9,2,Toallas y Toallones,2
10,2,Toallas y Toallones,2
11,2,Toallas y Toallones,2
12,2,Toallas y Toallones,2


Como puede ver, este conjunto de datos realmente no se parece a la matriz RR. Esto se debe a que faltan muchos valores, por lo que es mucho más fácil guardar el archivo en un formato disperso. En un formato disperso, la primera columna es el número de fila de la matriz ii; la segunda columna es el número de columna de la matriz jj; y la tercera fila es la entrada de matriz RijRij. Para este conjunto de datos, la primera columna es la ID de usuario, la segunda es la ID de la película que revisaron y la tercera columna es su puntaje de revisión (nosotros usaremos las productos y las compras de los mismos). Este formato disperso es también la entrada que requieren los métodos de factorización matricial, en lugar del RR de matriz completa, esto se debe a que solo usan las entradas de matriz no faltantes.

### Ajustando el modelo

Ahora es el momento de comenzar a usar el paquete. Primero necesitamos cargar el conjunto de datos en el paquete surprise, esto se hace usando la clase Reader. Lo principal que hace la clase Reader es especificar el rango de las revisiones. Primero verifiquemos el rango de las revisiones para este conjunto de datos.

In [4]:
lower_rating = dataset['rating'].min() 
upper_rating = dataset['rating'].max() 
print('Review range: {0} to {1} '.format(lower_rating, upper_rating)) 

Review range: 1 to 5 


Por lo tanto, nuestro rango de revisión va de 1 a 20, que es un poco no estándar (el valor predeterminado para surprise es 1-5). Por lo tanto, necesitaremos cambiar esto cuando carguemos nuestro conjunto de datos, que se hace así:

In [5]:
reader = surprise.Reader(rating_scale = (1.,5.))
data = surprise.Dataset.load_from_df(dataset, reader)

Usaremos el método SVD ++, uno de los mejores en el desafío de Netflix, que ahora se ha convertido en un método popular para ajustar los sistemas de recomendación. Como mencionamos anteriormente, este método extiende los algoritmos SVD de vainilla, solo optimizando los términos conocidos y realizando la regularización (tenga en cuenta que el método SVD en surprise es mucho más sofisticado que el SVD de vainilla, y mucho más similar a SVD ++). Un sistema de recomendación simple creado con SVD ++ se puede codificar de la siguiente manera:

In [6]:
alg = surprise.SVDpp() 
output = alg.fit(data.build_full_trainset()) 

Por ahora acabamos de entrenar el modelo en todo el conjunto de datos, lo cual no es una buena práctica, pero lo hacemos solo para darle una idea de cómo funcionan los modelos y las predicciones. Más adelante cubriremos las pruebas y evaluaciones adecuadas; así como el ajuste de hiperparámetros para maximizar el rendimiento.
Ahora que hemos ajustado el modelo, podemos verificar la puntuación pronosticada, por ejemplo, del usuario 2 en un producto 'Toallas y Toallones' usando el método de predicción.

In [7]:
# Los uids e iids deben establecerse como cadenas
pred = alg.predict(uid='2', iid='Toallas y Toallones')
score = pred.est 
print(score)

1.199334198351144


Entonces, en este caso, la estimación fue un puntaje de ... Pero para recomendar los mejores productos a los usuarios, necesitamos encontrar n elementos que tengan el puntaje más alto pronosticado. Haremos esto en la siguiente sección.


### Haciendo recomendaciones

Hagamos nuestras recomendaciones a un usuario en particular. Centrémonos en el uid 2 y busquemos un elemento para recomendarlos. Primero, necesitamos encontrar los identificadores de productos que el usuario 2 no calificó, ¡ya que no queremos recomendarles una producto que ya haya comprado!

In [8]:
# Obtenga una lista de todos los identificadores de productos 
iids = dataset['iid'].unique()
# Obtenga una lista de iids que uid 2 ha comprado
iids2 = dataset.loc[dataset['uid'] == 2, 'iid']
# Elimine los identificadores que uid 2 ha comprado de la lista de todos los identificadores de productos
iids_to_pred = np.setdiff1d(iids, iids2)

A continuación, queremos predecir la puntuación de cada uno de los identificadores de productos que el usuario 2 no compró, y encontrar el mejor. Para esto, tenemos que crear otro conjunto de datos con los iids que queremos predecir en el formato disperso como antes de: uid, iid, rating. Simplemente estableceremos arbitrariamente todas las calificaciones de este conjunto de pruebas en 4, ya que no son necesarias. Hagamos esto, luego produzca la primera predicción.

In [9]:
testset = [[2, iid, 4.] for iid in iids_to_pred]
predictions = alg.test(testset)
predictions[0] 

Prediction(uid=2, iid='Accesorios', r_ui=4.0, est=1.4044980069661965, details={'was_impossible': False})

Como puede ver en la salida, cada predicción es un objeto especial. Para encontrar lo mejor, convertiremos este objeto en una matriz de las calificaciones pronosticadas. Luego usaremos esto para encontrar el iid con la mejor calificación pronosticada.

In [10]:
pred_ratings = np.array([pred.est for pred in predictions])
# Encuentre el índice de la calificación máxima pronosticada 
i_max = pred_ratings.argmax() 
# Use esto para encontrar el iid correspondiente para recomendar
iid = iids_to_pred[i_max] 
print('Top item for user 2 has iid {0} with predicted rating {1}'.format(iid, pred_ratings[i_max]))

Top item for user 2 has iid Almohadas with predicted rating 2.085656519188416


### Ajuste y evaluación del modelo

Como probablemente ya sepa, es una mala práctica ajustar un modelo en todo el conjunto de datos sin verificar su rendimiento y los parámetros de ajuste que afectan el ajuste. Entonces, para el resto del tutorial, le mostraremos cómo ajustar los parámetros de SVD ++ y evaluar el rendimiento del método. El método SVD ++, así como la mayoría de los otros algoritmos de factorización de matriz, dependerán de una serie de constantes de sintonización principales: la dimensión DD que afecta el tamaño de UU y VV; la tasa de aprendizaje, que afecta el desempeño del paso de optimización; el término de regularización que afecta el sobreajuste del modelo; y el número de épocas, que determina cuántas iteraciones de optimización se utilizan.
En este tutorial ajustaremos la tasa de aprendizaje y el término de regularización. SVD ++ tiene más de una tasa de aprendizaje y término de regularización. Pero surprise permite que se establezca un valor fijo para todos los valores de velocidad de aprendizaje, y otro para todos los términos de regularización, por lo que haremos esto para la velocidad. Sorprendentemente, el ajuste se realiza mediante una función llamada GridSearchCV, que selecciona las constantes que funcionan mejor para predecir un conjunto de pruebas extendido. Esto significa que los valores constantes para probar deben estar predefinidos.
Primero definamos nuestra lista de valores constantes para verificar, por lo general, la tasa de aprendizaje es un valor pequeño entre 0 y 1. En teoría, el parámetro de regularización puede ser cualquier valor real positivo, pero en la práctica es limitado, ya que establecerlo demasiado pequeño resultará en sobreajuste, mientras que establecerlo demasiado grande dará como resultado un bajo rendimiento; entonces intentar una lista de valores razonables debería estar bien. La función GridSearchCV se puede utilizar para determinar los valores de los parámetros con mejor rendimiento mediante la validación cruzada. Hemos elegido una lista bastante limitada ya que este código puede tardar un tiempo en ejecutarse, ya que tiene que adaptarse a múltiples modelos con diferentes parámetros.


In [11]:
param_grid = {'lr_all' : [.001, .01], 'reg_all' : [.1, .5]} 
gs = surprise.model_selection.GridSearchCV(surprise.SVDpp, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit (data) 
# Imprimir combinación de parámetros que dieron la mejor puntuación RMSE 
print(gs.best_params['rmse'])

{'lr_all': 0.01, 'reg_all': 0.1}


La salida imprime la combinación de parámetros que obtienen el mejor RMSE en un conjunto de prueba extendido, RMSE es una forma de medir el error de predicción. En este caso, solo hemos verificado unos pocos valores constantes de ajuste, porque estos procedimientos pueden tardar un tiempo en ejecutarse. Pero, por lo general, probará tantos valores como sea posible para obtener el mejor rendimiento posible.
El rendimiento de un modelo particular que ha elegido puede evaluarse mediante validación cruzada. Esto podría usarse para comparar varios métodos, por ejemplo, o simplemente para verificar que su método esté funcionando razonablemente. Esto se puede hacer ejecutando lo siguiente:

In [12]:
alg = surprise.SVDpp(lr_all = .001) # Las opciones de parámetros se pueden agregar aquí.
output = surprise.model_selection.cross_validate(alg, data, verbose= True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.4724  0.4631  0.4593  0.4657  0.4636  0.4648  0.0043  
MAE (testset)     0.2629  0.2654  0.2612  0.2594  0.2587  0.2615  0.0024  
Fit time          252.84  247.97  248.55  247.91  249.23  249.30  1.83    
Test time         7.89    7.73    7.88    7.77    7.79    7.81    0.06    
